# Imports

In [ ]:
import math
import numpy as np
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
import random
import shutil
import os
import time

# Learning Models

In [ ]:
evalMetrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc')]

# FCN Model
def makeFCNModel():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (22, 33, 33)))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# CNN Model
def makeCNNModel():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = (22, 33, 33)))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation = 'relu'))#Dense
  model.add(tf.keras.layers.Dense(128, activation = 'relu'))#Dense
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# Select Training Data (requires benchmark.pkl)

In [ ]:
df = pd.read_pickle('benchmarkInfo/benchmark.pkl', compression='zip')
df = df.loc[df['FDRTotal'] == 0]# Get ONLY drv clean routed circuits, that have DRVs in their first DR itr.

circuitsToTrain = []
for x in df['Design']:
  density = int(x[x.rfind('_')+1:])
#   if density != 80 and density > 74 and density < 86: # density 80 is only for testing!
  if density == 80: # density 80 is only for testing!
    circuitsToTrain.append(x)
#   if density == 75: # test data augmentation strategy
#     circuitsToTrain.append(x)

allPkls = ['/data/CSV/' + x[:x.rfind('_')] + '/cts_' + x + '.pkl' for x in circuitsToTrain]
allPkls = [x for x in allPkls if os.path.exists(x)] #few cases where we don't the pkl, because first itr waas already drv clean

# Traning

In [ ]:
pos = 3327
neg = 876673
total = pos+neg
w0 = total/(2*neg)
w1 = total/(2*pos)
weights = {0: w0, 1: w1}
sizeBatch = 64

def train(pklsForTraining, learningModel, modelPath, epochStart, epochEnd, trainResultDF = pd.DataFrame()):
  pkls = pklsForTraining.copy()
  for epoch in range(epochStart, epochEnd):
    random.shuffle(pkls)
    for pkl in pkls:
      trainDf = pd.read_pickle(pkl, compression='zip')
      trainDf = trainDf.reset_index(drop=True)
      valDf = trainDf.sample(frac=0.2)
      trainDf = trainDf.drop(valDf.index)

      labels = trainDf.pop(trainDf.columns.values[-1])
      valLabels = valDf.pop(valDf.columns.values[-1])
      trainDf.pop(trainDf.columns.values[0])#drop first column which contains the nodeIds
      valDf.pop(valDf.columns.values[0])#drop first column which contains the nodeIds
      trainHyperImages = np.array(trainDf).reshape(len(trainDf),22,33,33)
      valHyperImages = np.array(valDf).reshape(len(valDf),22,33,33)
      print('Epoch: ',epoch,' Training with:', pkl)
      train_history = learningModel.fit(x=trainHyperImages,
                                       y=labels,
                                       verbose=2, #0 = silent, 1 = progress bar, 2 = one line per epoch
                                       batch_size=sizeBatch,
                                       validation_data=(valHyperImages, valLabels),
                                       class_weight=weights)
      historyDf = pd.DataFrame(train_history.history)
      historyDf['epoch'] = epoch
      historyDf['design'] = pkl[pkl.rfind('/')+5:pkl.find('.')]
      trainResultDF = pd.concat([trainResultDF, historyDf])
    pickle.dump(learningModel, open(modelPath+'model_'+str(epoch)+'.pkl', 'wb'))
    pickle.dump(trainResultDF, open(modelPath+'trainResultDF.pkl', 'wb'))

In [ ]:
modelPath = 'results/cnn/'
useFCN = False

In [ ]:
modelPath = 'results/fcnNoOversampling/'
useFCN = True

In [ ]:
numEpochs = 50

if os.path.exists(modelPath) == False:
  os.mkdir(modelPath)

models = [x for x in os.listdir(modelPath)]
lastRunEpoch = 0
learningModel = None
trainResultDF = pd.DataFrame()
if len(models) > 0:
  if 'trainResultDF.pkl' in models:
    models.remove('trainResultDF.pkl')
  models.sort(key = lambda x : int(x[x.find('_')+1:x.find('.')]))
  lastModel = models[-1]
  lastRunEpoch = int(lastModel[lastModel.find('_')+1:lastModel.find('.')])
  learningModel = pickle.load(open(modelPath+'model_'+str(lastRunEpoch)+'.pkl', 'rb'))
  lastRunEpoch += 1
  trainResultDF = pickle.load(open(modelPath+'trainResultDF.pkl', 'rb'))
else:
  learningModel = makeFCNModel() if useFCN else makeCNNModel()

train(allPkls, learningModel, modelPath, lastRunEpoch, numEpochs, trainResultDF)

# Cross Validation

In [ ]:
def predict(model, pkl):
  testDf = pd.read_pickle(pkl, compression='zip')
  labels = testDf.pop(testDf.columns.values[-1])
  testDf.pop(testDf.columns.values[0])#drop first column which contains the nodeIds
  testHyperImages = np.array(testDf).reshape(len(testDf),22,33,33)
  result = model.evaluate(testHyperImages, labels)
  resultDict = {m:r for (m, r) in zip(model.metrics_names, result)}
  return resultDict


# modelsToCrossValidate = ['results/fcn/model_0.pkl', 'results/fcn/model_15.pkl', 'results/fcn/model_25.pkl']
modelsToCrossValidate = ['results/fcnNoOversampling/model_18.pkl']
circuitsToTest = ['/data/CSVWhole/'+x+'/' for x in os.listdir('/data/CSVWhole/')]

for modelPath in modelsToCrossValidate:
  model = pickle.load(open(modelPath, 'rb'))
#   outputFile = 'results/cv/fcnCV'+modelPath[modelPath.find('_')+1:modelPath.find('.')]+'.pkl'
  outputFile = 'results/cv/fcnNoOversampling.pkl'
  resultDF = pd.DataFrame()
  for circuit in circuitsToTest:
    pkls = [x for x in os.listdir(circuit) if '.pkl' in x]
    pkls.sort(key = lambda x : int(x[x.rfind('_')+1:x.find('.')]))
    itrDf = pd.DataFrame()
    for pkl in pkls:
      resultDict = predict(model, circuit+pkl)
      resultDf = pd.DataFrame(resultDict, index=[0])
      itrDf = pd.concat([itrDf, resultDf], ignore_index=True)
    itrDf = itrDf.sum(axis=0)
    itrDf['Design'] = pkls[0][pkls[0].find('_')+1:pkls[0].rfind('_')]
    itrDf = pd.DataFrame(itrDf).T
    resultDF = pd.concat([resultDF, itrDf], ignore_index=True)
  pickle.dump(resultDF, open(outputFile, 'wb'))

# Benchmark Info (benchmark.pkl)

# Compress and merge CSVs into .pkls